In [107]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np

In [3]:
file = '../mini-mnist/mnist-00000-of-00002.tfrecord'
os.path.isfile(file)

True

# Two ways to iterate a SequenceExample proto

## Method 1: using `tf.data.TFRecordDataset`

In [4]:
dataset = tf.data.TFRecordDataset(file)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [5]:
with tf.Session() as sess:
  print(sess.run(next_element))

b'\n/\n\x17\n\x0blabel_score\x12\x08\x12\x06\n\x04\x00\x00\x80?\n\x14\n\x0blabel_index\x12\x05\x1a\x03\n\x01\x05\x12\xde\x18\n\xdb\x18\n\r0_dense_input\x12\xc9\x18\n\xc6\x18\x12\xc3\x18\n\xc0\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Method 2: using `tf.python_io.tf_record_iterator`

In [6]:
path_to_tfrecord = file
context_features={"label_index": tf.VarLenFeature(tf.int64),
                  "label_score": tf.VarLenFeature(tf.float32)
                 }
key_dense = '0_dense_input'
# sequence_features = {key_dense: tf.FixedLenSequenceFeature((28,28), dtype=tf.float32)}
sequence_features = {key_dense: tf.VarLenFeature(dtype=tf.float32)}


for sequence_example in tf.python_io.tf_record_iterator(path_to_tfrecord):
    print('Before parsing, we only have unreadable bytes:\n', sequence_example)
    contexts, features =\
      tf.parse_single_sequence_example(sequence_example, 
                                       context_features=context_features,
                                       sequence_features=sequence_features)
    break

Before parsing, we only have unreadable bytes:
 b'\n/\n\x17\n\x0blabel_score\x12\x08\x12\x06\n\x04\x00\x00\x80?\n\x14\n\x0blabel_index\x12\x05\x1a\x03\n\x01\x05\x12\xde\x18\n\xdb\x18\n\r0_dense_input\x12\xc9\x18\n\xc6\x18\x12\xc3\x18\n\xc0\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [7]:
features

{'0_dense_input': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x113339828>}

In [8]:
contexts

{'label_index': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x1132cc7b8>,
 'label_score': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x113339630>}

In [9]:
with tf.Session() as sess:
  res = sess.run(features[key_dense])
  print(res)

SparseTensorValue(indices=array([[  0,   0],
       [  0,   1],
       [  0,   2],
       ...,
       [  0, 781],
       [  0, 782],
       [  0, 783]]), values=array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.  

In [18]:
res.indices.shape

(784, 2)

In [21]:
res.indices

array([[  0,   0],
       [  0,   1],
       [  0,   2],
       ...,
       [  0, 781],
       [  0, 782],
       [  0, 783]])

In [20]:
res.values

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [23]:
res.values.shape

(784,)

In [ ]:
with tf.Session() as sess:
  print(contexts['label_index'].eval())

In [ ]:
with tf.Session() as sess:
  values = sess.run(contexts['label_index'].values)
  indices = sess.run(contexts['label_index'].indices)
  print(value, indices)

# Two ways to parse a SequenceExample proto

## Method 1: use `tf.train.SequenceExample.FromString`

In [10]:
for sequence_example in tf.python_io.tf_record_iterator(path_to_tfrecord):
    parsed_sequence_example = tf.train.SequenceExample.FromString(sequence_example)
    print(parsed_sequence_example)
    break

context {
  feature {
    key: "label_index"
    value {
      int64_list {
        value: 5
      }
    }
  }
  feature {
    key: "label_score"
    value {
      float_list {
        value: 1.0
      }
    }
  }
}
feature_lists {
  feature_list {
    key: "0_dense_input"
    value {
      feature {
        float_list {
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
          value: 0.0
     

In [17]:
import inspect

inspect.getmembers(parsed_sequence_example)

[('ByteSize', <function SequenceExample.ByteSize>),
 ('CONTEXT_FIELD_NUMBER', 1),
 ('Clear', <function SequenceExample.Clear>),
 ('ClearExtension', <function SequenceExample.ClearExtension>),
 ('ClearField', <function SequenceExample.ClearField>),
 ('CopyFrom', <function SequenceExample.CopyFrom>),
 ('DESCRIPTOR',
  <google.protobuf.pyext._message.MessageDescriptor at 0x11153c4d0>),
 ('DiscardUnknownFields', <function SequenceExample.DiscardUnknownFields>),
 ('FEATURE_LISTS_FIELD_NUMBER', 2),
 ('FindInitializationErrors',
  <function SequenceExample.FindInitializationErrors>),
 ('FromString', <function SequenceExample.FromString>),
 ('HasExtension', <function SequenceExample.HasExtension>),
 ('HasField', <function SequenceExample.HasField>),
 ('IsInitialized', <function SequenceExample.IsInitialized>),
 ('ListFields', <function SequenceExample.ListFields>),
 ('MergeFrom', <function SequenceExample.MergeFrom>),
 ('MergeFromString', <function SequenceExample.MergeFromString>),
 ('ParseFr

In [33]:
context_feature = parsed_sequence_example.context.feature
for x in context_feature:
  y = context_feature[x]

In [47]:
parsed_sequence_example.context

feature {
  key: "label_index"
  value {
    int64_list {
      value: 5
    }
  }
}
feature {
  key: "label_score"
  value {
    float_list {
      value: 1.0
    }
  }
}

In [94]:
feature = parsed_sequence_example.context.feature['label_index']

In [104]:
type(feature_list)

tensorflow.core.example.feature_pb2.FeatureList

In [103]:
type(feature_list.feature[0])

tensorflow.core.example.feature_pb2.Feature

In [106]:
import pandas as pd
import numpy as np

In [ ]:
pd.Series(np)

In [105]:
list(getattr(feature, 'int64_list').value)

NameError: name 'avg' is not defined

In [96]:
feature

TypeError: 'Feature' object is not subscriptable

In [95]:
hasattr(feature, 'int64_list')

True

In [109]:
haha = pd.Series(range(10))

In [111]:
haha.max()

9

In [65]:
classes = set()

In [69]:
classes.add('haha')

In [74]:
haha = [].append('haha')

In [71]:
classes.union(classes)

{'haha'}

In [78]:
feature_list = parsed_sequence_example.feature_lists.feature_list

In [83]:
feature_list

google.protobuf.pyext._message.MessageMapContainer

In [84]:
type(parsed_sequence_example.feature_lists)

tensorflow.core.example.feature_pb2.FeatureLists

In [85]:
feature_list = parsed_sequence_example.feature_lists.feature_list['0_dense_input']

In [87]:
type(feature_list)

tensorflow.core.example.feature_pb2.FeatureList

In [90]:
len(feature_list.feature)

1

In [80]:
for x in feature_list:
  print(feature_list[x])

feature {
  float_list {
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0


In [52]:
list(parsed_sequence_example.feature_lists.feature_list)

['0_dense_input']

In [53]:
parsed_sequence_example.feature_lists.feature_list['0_dense_input']

feature {
  float_list {
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0
    value: 0.0


In [41]:
for key in parsed_sequence_example.feature_lists.feature_list:
  print(key)

0_dense_input


In [44]:
True \
and False

False

In [38]:
len(y.float_list.value)

1

In [28]:
parsed_sequence_example

TypeError: argument of type 'SequenceExample' is not iterable

In [20]:
parsed_sequence_example.context.

AttributeError: 'Features' object has no attribute 'label_index'

In [ ]:
parsed_sequence_example.context

In [ ]:
parsed_sequence_example.feature_lists

## Method 2: use `tf.parse_single_sequence_example` 

### (see Method 2 in the previous section) 